# Quantum Approximate Optimization Algorithm

In this notebook we are going to show how to use the implementation of QAOA available in Aqua to obtain solutions to the MaxCut problem

In [1]:
import numpy as np

from qiskit import Aer, IBMQ
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import *
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.providers.aer.noise import NoiseModel

provider = IBMQ.load_account()

/opt/conda/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


First, we define a function that from the coefficients of an Ising model creates the Hamiltonian for which we are going to find the ground state.

In [2]:
def get_operator(J,h,n):    
    pauli_list = []

    for (i,j) in J: # For each coefficient in J (couplings) we add a term J[i,j]Z_iZj
        x_p = np.zeros(n, dtype=np.bool)
        z_p = np.zeros(n, dtype=np.bool)
        z_p[n-1-i] = True 
        z_p[n-1-j] = True
        pauli_list.append([J[(i,j)],Pauli(z_p, x_p)])
     
    for i in h: # For each coefficient in h we add a term h[i]Z_i
        x_p = np.zeros(n, dtype=np.bool)
        z_p = np.zeros(n, dtype=np.bool)
        z_p[n-1-i] = True
        pauli_list.append([h[i],Pauli(z_p, x_p)])
    
    return WeightedPauliOperator(paulis=pauli_list)

Now, we define the edges of the graph and obtain the Hamiltonian. For this graph, which is a cycle of length 5, the optimal solution gives a cost of -3

In [3]:
# Edges of the graph

J1 = {(0,1):1, (1,2):1, (2,3):1, (3,4):1, (4,0):1}
h1 = {}
n = 5

# Hamiltonian

q_op =get_operator(J1,h1,n) 
print(q_op)
q_op.print_details()

Representation: paulis, qubits: 5, size: 5


'ZZIII\t(1+0j)\nIZZII\t(1+0j)\nIIZZI\t(1+0j)\nIIIZZ\t(1+0j)\nZIIIZ\t(1+0j)\n'

We are going to run 10 repetitions on the statevector simulator

In [4]:
rep = 10
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)

We run QAOA with COBYLA as the classical optimizer and with optimization level $p = 1$

In [5]:
p = 1
val = 0
for i in range(rep):
    print("----- ITERATION ",i, " ------")
    optimizer = COBYLA()
    qaoa = QAOA(q_op, optimizer, p=p)
    result = qaoa.run(quantum_instance)
    print("Optimal value", result['optimal_value'])
    val+=result['optimal_value']
print("----- AVERAGE -----")
print("Average value",val/rep)

----- ITERATION  0  ------
Optimal value -2.499999944131867
----- ITERATION  1  ------
Optimal value -2.499999944131867
----- ITERATION  2  ------
Optimal value -2.499999944131867
----- ITERATION  3  ------
Optimal value -2.499999944131867
----- ITERATION  4  ------
Optimal value -2.499999944131867
----- ITERATION  5  ------
Optimal value -2.499999944131867
----- ITERATION  6  ------
Optimal value -2.499999944131867
----- ITERATION  7  ------
Optimal value -2.499999944131867
----- ITERATION  8  ------
Optimal value -2.499999944131867
----- ITERATION  9  ------
Optimal value -2.499999944131867
----- AVERAGE -----
Average value -2.4999999441318677


Now, we increase $p$ to $2$

In [6]:
p = 2
val = 0
for i in range(rep):
    print("----- ITERATION ",i, " ------")
    optimizer = COBYLA()
    qaoa = QAOA(q_op, optimizer, p=p)
    result = qaoa.run(quantum_instance)
    print("Optimal value", result['optimal_value'])
    val+=result['optimal_value']
print("----- AVERAGE -----")
print("Average value",val/rep)

----- ITERATION  0  ------
Optimal value -2.9999850752772828
----- ITERATION  1  ------
Optimal value -2.9999850752772828
----- ITERATION  2  ------
Optimal value -2.9999850752772828
----- ITERATION  3  ------
Optimal value -2.9999850752772828
----- ITERATION  4  ------
Optimal value -2.9999850752772828
----- ITERATION  5  ------
Optimal value -2.9999850752772828
----- ITERATION  6  ------
Optimal value -2.9999850752772828
----- ITERATION  7  ------
Optimal value -2.9999850752772828
----- ITERATION  8  ------
Optimal value -2.9999850752772828
----- ITERATION  9  ------
Optimal value -2.9999850752772828
----- AVERAGE -----
Average value -2.999985075277283


We are going to run the algorithm with a backend which includes a noise model

In [7]:
rep = 10
backendIBM = provider.get_backend('ibmq_ourense')
noise_model = NoiseModel.from_backend(backendIBM)
coupling_map = backendIBM.configuration().coupling_map
basis_gates = noise_model.basis_gates
backend = Aer.get_backend("qasm_simulator")


shots = 8192
optimization_level = 3
p = 1
quantum_instance = QuantumInstance(backend, shots = shots, 
                                    optimization_level = optimization_level,
                                    noise_model = noise_model,
                                    basis_gates = basis_gates,
                                    coupling_map = coupling_map)

In [8]:
p = 1
val = 0
for i in range(rep):
    print("----- ITERATION ",i, " ------")
    optimizer = COBYLA()
    qaoa = QAOA(q_op, optimizer, p=p)
    result = qaoa.run(quantum_instance)
    print("Optimal value", result['optimal_value'])
    val+=result['optimal_value']
print("----- AVERAGE -----")
print("Average value",val/rep)

----- ITERATION  0  ------
Optimal value -1.96728515625
----- ITERATION  1  ------
Optimal value -1.95068359375
----- ITERATION  2  ------
Optimal value -1.91943359375
----- ITERATION  3  ------
Optimal value -1.90234375
----- ITERATION  4  ------
Optimal value -1.93115234375
----- ITERATION  5  ------
Optimal value -1.94384765625
----- ITERATION  6  ------
Optimal value -1.95068359375
----- ITERATION  7  ------
Optimal value -1.90283203125
----- ITERATION  8  ------
Optimal value -1.9091796875
----- ITERATION  9  ------
Optimal value -1.92138671875
----- AVERAGE -----
Average value -1.9298828125
